# DataFrame

http://spark.apache.org/docs/2.3.1/api/python/pyspark.sql.html#pyspark-sql-module

In [31]:
import pyspark
import os

from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession.builder \
    .appName('DataFrame') \
    .master('local[*]') \
    .getOrCreate()

os.environ['PYSPARK_PYTHON'] = '/opt/anaconda3/bin/python3'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'ipython3'

## Tworzenie DataFrame'u

### Plik

In [32]:
data_path = './SparkSQLdata/'

In [33]:
# json
people = spark.read.json(data_path+'people.json')

In [34]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
+----+-------+



In [35]:
# csv
people_txt = spark.read.option("inferSchema", "true").csv(data_path+'people.txt')

In [36]:
people_txt.show()

+-------+----+
|    _c0| _c1|
+-------+----+
|Michael|29.0|
|   Andy|30.0|
| Justin|19.0|
+-------+----+



### Kolekcja Row'ów

In [37]:
newPerson1 = Row(name='Greg', age=32)

In [38]:
newPerson1

Row(age=32, name='Greg')

In [39]:
newPerson1.name

'Greg'

In [40]:
newPerson1.age

32

In [41]:
newPerson1['age']

32

In [42]:
'age' in newPerson1

True

In [43]:
newPerson = Row("age", "name")

In [44]:
newPerson2 = newPerson(24, 'Alice')

In [45]:
newPerson2

Row(age=24, name='Alice')

In [46]:
newPerson3 = newPerson(None, None)
newPerson4 = newPerson(33, None)
newPerson5 = newPerson(None, 'Peter')
newPerson6 = newPerson(32, 'Peter')
newPerson7 = newPerson(40, 'Greg')

In [47]:
newPeopleDF = spark.createDataFrame([newPerson1, newPerson2, newPerson3, newPerson4, 
                                     newPerson5, newPerson6, newPerson7])

In [48]:
newPeopleDF.show()

+----+-----+
| age| name|
+----+-----+
|  32| Greg|
|  24|Alice|
|null| null|
|  33| null|
|null|Peter|
|  32|Peter|
|  40| Greg|
+----+-----+



### Inne lokalne kolekcje

Typy danych: http://spark.apache.org/docs/2.3.1/api/python/pyspark.sql.html#module-pyspark.sql.types

Kilka podstawowych: IntegerType, DoubleType, FloatType, StringType, BooleanType, NullType

In [49]:
from pyspark.sql.types import IntegerType, StringType, StructType, StructField

In [50]:
# definicja schematu
# StructType ~ Row
schema = StructType([StructField("V1", IntegerType()), StructField("V2", StringType())])

In [51]:
# lokalna kolekcja - lista list
df = spark.createDataFrame([[1,2],[3,4]], schema)

In [52]:
df.show()

+---+---+
| V1| V2|
+---+---+
|  1|  2|
|  3|  4|
+---+---+



### RDD

Przechodzenie RDD <-> DF

In [53]:
type(people)

pyspark.sql.dataframe.DataFrame

In [54]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
+----+-------+



In [55]:
people.collect()

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin'),
 Row(age=35, name='Emma'),
 Row(age=None, name=None),
 Row(age=31, name=None)]

DF -> RDD

In [56]:
type(people.rdd)

pyspark.rdd.RDD

In [57]:
people.rdd.collect()

[Row(age=None, name='Michael'),
 Row(age=30, name='Andy'),
 Row(age=19, name='Justin'),
 Row(age=35, name='Emma'),
 Row(age=None, name=None),
 Row(age=31, name=None)]

In [58]:
people.rdd.map(tuple).collect()

[(None, 'Michael'),
 (30, 'Andy'),
 (19, 'Justin'),
 (35, 'Emma'),
 (None, None),
 (31, None)]

RDD -> DF

In [59]:
people.rdd.toDF().show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
+----+-------+



In [60]:
people.rdd.map(tuple).toDF().show()

+----+-------+
|  _1|     _2|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
+----+-------+



In [61]:
sc = spark.sparkContext

In [62]:
sc.parallelize([(x, x+1) for x in range(5)]).toDF().show()

+---+---+
| _1| _2|
+---+---+
|  0|  1|
|  1|  2|
|  2|  3|
|  3|  4|
|  4|  5|
+---+---+



In [75]:
# do toDF można podać schemat
schema = StructType([StructField("A", IntegerType()), StructField("B", StringType())])
sc.parallelize([(x, x+1) for x in range(5)]).toDF(schema).show()

+---+---+
|  A|  B|
+---+---+
|  0|  1|
|  1|  2|
|  2|  3|
|  3|  4|
|  4|  5|
+---+---+



In [76]:
sc.parallelize([(x, x+1) for x in range(5)]).toDF(schema).collect()[0]

Row(A=0, B='1')

> **TODO**: Na dwa sposoby stwórz DataFrame z 3 wierszami i 3 kolumnami - dwie typu string, jedna numeryczna

In [84]:
schema1 = StructType([StructField("S1", StringType()), StructField("S2", StringType()), StructField("I1", IntegerType())])
sc.parallelize([(x, x+1, x+2) for x in range(3)]).toDF(schema1).show()

+---+---+---+
| S1| S2| I1|
+---+---+---+
|  0|  1|  2|
|  1|  2|  3|
|  2|  3|  4|
+---+---+---+



In [94]:
constr = Row("s1","s2","i1")
r1 = constr("a","b",2)
r2 = constr("c","D",3)
r3 = constr("E","f",5)
spark.createDataFrame([r1,r2,r3]).show()

+---+---+---+
| s1| s2| i1|
+---+---+---+
|  a|  b|  2|
|  c|  D|  3|
|  E|  f|  5|
+---+---+---+



> **TODO**: Z podanego RDD utwórz DataFrame z nazwanymi kolumnami `name` i `age` oraz odpowiednimi typami (string i int)

In [95]:
myRDD = sc.textFile(data_path+"people.txt")
myRDD.collect()

['Michael, 29', 'Andy, 30', 'Justin, 19']

In [101]:
myRDD.map(lambda x: x.split(", ")).collect()
schema2 = StructType([StructField("Name", StringType()), StructField("Age", IntegerType())])

In [104]:
## wszystko w stringach
#myRDD.map(lambda x: x.split(", ")).toDF().printSchema()

myRDD.map(lambda x: x.split(", ")).map(lambda x: (x[0], int(x[1]))).toDF(schema2).printSchema()


root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)



****

## Praca z DataFrame'ami

### Ogolne wiadomosci na temat danych
printSchema, show, columns, dtypes <br>
Znane z RDD np: count, take, head

In [66]:
people.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [67]:
people_txt.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: double (nullable = true)



In [68]:
people_txt.show(1)

+-------+----+
|    _c0| _c1|
+-------+----+
|Michael|29.0|
+-------+----+
only showing top 1 row



In [105]:
people.show(1)

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
+----+-------+
only showing top 1 row



Liczba wierszy

In [106]:
people.count()

6

Lista kolumn

In [107]:
people.columns

['age', 'name']

Lista kolumn wraz z typami danych

In [108]:
newPeopleDF.dtypes

[('age', 'bigint'), ('name', 'string')]

### Odwolania do poszczegolnych kolumn

In [109]:
people.age

Column<b'age'>

In [110]:
people['age']

Column<b'age'>

In [111]:
people[0]

Column<b'age'>

### Dodanie/usuniecie kolumny
withColumn, drop

In [112]:
people.withColumn(colName = 'ageNextYear', col = people.age +1).show()

+----+-------+-----------+
| age|   name|ageNextYear|
+----+-------+-----------+
|null|Michael|       null|
|  30|   Andy|         31|
|  19| Justin|         20|
|  35|   Emma|         36|
|null|   null|       null|
|  31|   null|         32|
+----+-------+-----------+



In [113]:
people.drop('age').show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
|   Emma|
|   null|
|   null|
+-------+



### Zmiany nazwy kolumny

In [115]:
people_txt = people_txt.withColumnRenamed('_c0', 'name').withColumnRenamed('_c1', 'age')

In [116]:
people_txt.show()

+-------+----+
|   name| age|
+-------+----+
|Michael|29.0|
|   Andy|30.0|
| Justin|19.0|
+-------+----+



### Podstawowe statystyki kolumn w DataFrame'ie.
describe

In [117]:
people.describe().show()

+-------+-----------------+-------+
|summary|              age|   name|
+-------+-----------------+-------+
|  count|                4|      4|
|   mean|            28.75|   null|
| stddev|6.849574196011505|   null|
|    min|               19|   Andy|
|    max|               35|Michael|
+-------+-----------------+-------+



In [118]:
people.describe('age').show()

+-------+-----------------+
|summary|              age|
+-------+-----------------+
|  count|                4|
|   mean|            28.75|
| stddev|6.849574196011505|
|    min|               19|
|    max|               35|
+-------+-----------------+



### Braki danych
isNull, isNotNull<br>
fillna, dropna, replace

In [119]:
newPeopleDF.show()

+----+-----+
| age| name|
+----+-----+
|  32| Greg|
|  24|Alice|
|null| null|
|  33| null|
|null|Peter|
|  32|Peter|
|  40| Greg|
+----+-----+



In [120]:
newPeopleDF.filter(newPeopleDF.age.isNull()).show()

+----+-----+
| age| name|
+----+-----+
|null| null|
|null|Peter|
+----+-----+



In [121]:
newPeopleDF.filter(newPeopleDF.age.isNotNull()).show()

+---+-----+
|age| name|
+---+-----+
| 32| Greg|
| 24|Alice|
| 33| null|
| 32|Peter|
| 40| Greg|
+---+-----+



In [122]:
newPeopleDF.fillna(-1).show()

+---+-----+
|age| name|
+---+-----+
| 32| Greg|
| 24|Alice|
| -1| null|
| 33| null|
| -1|Peter|
| 32|Peter|
| 40| Greg|
+---+-----+



In [123]:
newPeopleDF.fillna({'age':-1, 'name':'unknown'}).show()

+---+-------+
|age|   name|
+---+-------+
| 32|   Greg|
| 24|  Alice|
| -1|unknown|
| 33|unknown|
| -1|  Peter|
| 32|  Peter|
| 40|   Greg|
+---+-------+



In [124]:
newPeopleDF.fillna({'name':'unknown'}).replace('unknown', 'NN').show()

+----+-----+
| age| name|
+----+-----+
|  32| Greg|
|  24|Alice|
|null|   NN|
|  33|   NN|
|null|Peter|
|  32|Peter|
|  40| Greg|
+----+-----+



In [125]:
newPeopleDF.dropna().show()

+---+-----+
|age| name|
+---+-----+
| 32| Greg|
| 24|Alice|
| 32|Peter|
| 40| Greg|
+---+-----+



In [126]:
newPeopleDF.dropna(subset='age').show()

+---+-----+
|age| name|
+---+-----+
| 32| Greg|
| 24|Alice|
| 33| null|
| 32|Peter|
| 40| Greg|
+---+-----+



### Funkcje wprost ze skladni SQL 
select, where (alias filter), orderBy

In [127]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
+----+-------+



In [128]:
people.\
select('name', 'age').\
where(people.name.like('%n%')).\
orderBy(people.age.asc()).\
show()
# select: wyswietl kolumny 'name' i 'age'
# where: uwzglednij tylko imiona (name) zaweirajace litere 'n'
# orderBy: zbior posortuj rosnaco po kolumnie 'age'

+------+---+
|  name|age|
+------+---+
|Justin| 19|
|  Andy| 30|
+------+---+



In [129]:
# gdy chcemy zobaczyć wszystkie kolumny select jest zbędny
people.\
where(people.name.like('%n%')).\
orderBy(people.age.asc()).\
show()

+---+------+
|age|  name|
+---+------+
| 19|Justin|
| 30|  Andy|
+---+------+



> **TODO**: Wyswietl imiona ludzi ze zbioru `people` starszych niz 29 lat. Wyniki posortuj alfabetycznie po kolumnie name

### Operacje na zbiorach
union - dziala jak UNION ALL w SQL. <br>
intersect (INTERSECT z SQLa), subtract (EXCEPT z SQLa)

In [130]:
people.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|  30|   Andy|
|  19| Justin|
|  35|   Emma|
|null|   null|
|  31|   null|
+----+-------+



In [131]:
people_txt.show()

+-------+----+
|   name| age|
+-------+----+
|Michael|29.0|
|   Andy|30.0|
| Justin|19.0|
+-------+----+



In [132]:
people.union(people_txt).show()

+-------+-------+
|    age|   name|
+-------+-------+
|   null|Michael|
|     30|   Andy|
|     19| Justin|
|     35|   Emma|
|   null|   null|
|     31|   null|
|Michael|   29.0|
|   Andy|   30.0|
| Justin|   19.0|
+-------+-------+



In [133]:
people.union(people_txt.select(people.columns)).show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|30.0|   Andy|
|19.0| Justin|
|35.0|   Emma|
|null|   null|
|31.0|   null|
|29.0|Michael|
|30.0|   Andy|
|19.0| Justin|
+----+-------+



###############################################################################################################

# Zadania 1

****
Przygotowanie danych

In [134]:
data_path = './SparkSQLdata/'

In [135]:
people = spark.read.json(data_path+'people.json')
employees = spark.read.json(data_path+'employees.json')
people_txt = spark.read.option("inferSchema", "true").csv(data_path+'people.txt')

In [136]:
newPerson1 = Row(name=u'Greg', age=32)
newPerson = Row("age", "name")
newPerson2 = newPerson(24, 'Alice')
newPerson3 = newPerson(None, None)
newPerson4 = newPerson(33, None)
newPerson5 = newPerson(None, 'Peter')
newPerson6 = newPerson(32, 'Peter')
newPerson7 = newPerson(40, 'Greg')

In [137]:
newPeopleDF = spark.createDataFrame([newPerson1, newPerson2, newPerson3, newPerson4, 
                                     newPerson5, newPerson6, newPerson7])

In [138]:
people_txt = people_txt.withColumnRenamed('_c0', 'name').withColumnRenamed('_c1', 'age')

In [139]:
allPeople = spark.read.parquet(data_path+'allPeople.parquet')

****

1. Wyswietl imiona ludzi ze zbioru `people`, o ktorych wielku nie mamy informacji.

In [141]:
people.where(people.age.isNull()).select("name").show()

+-------+
|   name|
+-------+
|Michael|
|   null|
+-------+



2. Na koniec zbioru `people` doklej zbiór `newPeopleDF` oraz `people_txt`. Tak utworzony DataFrame nazwij `allPeople`

In [142]:
cols = people.columns

In [144]:
cols

['age', 'name']

In [148]:
allPeople = people.union(newPeopleDF.select(cols)).union(people_txt.select(cols))
allPeople.show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|30.0|   Andy|
|19.0| Justin|
|35.0|   Emma|
|null|   null|
|31.0|   null|
|32.0|   Greg|
|24.0|  Alice|
|null|   null|
|33.0|   null|
|null|  Peter|
|32.0|  Peter|
|40.0|   Greg|
|29.0|Michael|
|30.0|   Andy|
|19.0| Justin|
+----+-------+



3. Wybierz ze zbioru `allPeople` te wiersze, ktore wystepuja rowniez w zbiorze `people`.

+----+-------+
| age|   name|
+----+-------+
|32.0|  Peter|
|24.0|  Alice|
|29.0|Michael|
|40.0|   Greg|
|33.0|   null|
|null|  Peter|
|32.0|   Greg|
+----+-------+



4. Wybierz ze zbioru `allPeople` te imiona, ktore NIE wystepuja w zbiorze `people_txt`.

In [159]:
allPeople.select("name").subtract(people_txt.select("name")).dropna().show()

+-----+
| name|
+-----+
| Greg|
|Alice|
| Emma|
|Peter|
+-----+



5. Usun ze zbioru `allPeople` wiersze, ktore zawieraja same braki danych.

In [164]:
allPeople.dropna("all").show()

+----+-------+
| age|   name|
+----+-------+
|null|Michael|
|30.0|   Andy|
|19.0| Justin|
|35.0|   Emma|
|31.0|   null|
|32.0|   Greg|
|24.0|  Alice|
|33.0|   null|
|null|  Peter|
|32.0|  Peter|
|40.0|   Greg|
|29.0|Michael|
|30.0|   Andy|
|19.0| Justin|
+----+-------+



6. Do zbioru `allPeople` dodaj kolumne 'age' zawierajaca wiek powiekszony o 1. Zmien nazwe oryginalnej kolumny 'age' na 'starting_age'.

In [167]:
allPeople = allPeople.withColumnRenamed("age","starting_age")
allPeople.withColumn("age", allPeople.starting_age + 1)
allPeople.show()

+------------+-------+
|starting_age|   name|
+------------+-------+
|        null|Michael|
|        30.0|   Andy|
|        19.0| Justin|
|        35.0|   Emma|
|        null|   null|
|        31.0|   null|
|        32.0|   Greg|
|        24.0|  Alice|
|        null|   null|
|        33.0|   null|
|        null|  Peter|
|        32.0|  Peter|
|        40.0|   Greg|
|        29.0|Michael|
|        30.0|   Andy|
|        19.0| Justin|
+------------+-------+



7. Ile unikatowych rekordow znajduje sie w zbiorze `allPeople`?

In [170]:
allPeople.distinct().count()

13

8. **⋆** Do zbioru `employees` dodaj kolumny: z pensjami po 0-10% podwyżce (dla każdego losowo z rozkładu jednostajnego) oraz z różnicą pomiędzy pensją przed i po podwyżce. Zbiór posortuj alfabetycznie.

In [173]:
from pyspark.sql import functions as f


In [171]:
employees.show()

+-------+------+
|   name|salary|
+-------+------+
|Michael|  3000|
|   Andy|  4500|
| Justin|  3500|
|  Berta|  4000|
+-------+------+



In [175]:
employees.withColumn("salar_inc", employees.salary * (f.rand(123)/10 + 1))\
.withColumn("salary_diff", employees.salary * f.rand(123)/10).orderBy("name").show()

+-------+------+------------------+------------------+
|   name|salary|         salar_inc|       salary_diff|
+-------+------+------------------+------------------+
|   Andy|  4500| 4944.037338866702| 444.0373388667023|
|  Berta|  4000| 4335.627127745331|335.62712774533077|
| Justin|  3500|3787.3371536584673| 287.3371536584678|
|Michael|  3000|3150.8860324144957| 150.8860324144958|
+-------+------+------------------+------------------+



###############################################################################################################

## Praca z DataFrame'ami

In [233]:
data_path = './SparkSQLdata/'
people = spark.read.json(data_path+'people.json')
employees = spark.read.json(data_path+'employees.json')
people_txt = spark.read.option("inferSchema", "true").csv(data_path+'people.txt')
newPeople = spark.read.parquet(data_path+'newPeople.parquet')
people_txt = people_txt.withColumnRenamed('_c0', 'name').withColumnRenamed('_c1', 'age')

### Funkcje SparkSQL

In [178]:
from pyspark.sql import functions as f

In [179]:
people.select(f.min("age")).show()

+--------+
|min(age)|
+--------+
|      19|
+--------+



In [180]:
people.select(f.min("age").alias("min_age")).show()

+-------+
|min_age|
+-------+
|     19|
+-------+



In [181]:
people.withColumn("random", f.randn(42)).show()

+----+-------+-------------------+
| age|   name|             random|
+----+-------+-------------------+
|null|Michael| 0.4085363219031828|
|  30|   Andy| 0.8811793095417685|
|  19| Justin| -2.013921870967947|
|  35|   Emma| 1.6641751435679302|
|null|   null|-1.0878600404148453|
|  31|   null| 1.1432831717404852|
+----+-------+-------------------+



In [182]:
people.withColumn("random", f.exp("age")).show()

+----+-------+--------------------+
| age|   name|              random|
+----+-------+--------------------+
|null|Michael|                null|
|  30|   Andy|1.068647458152446...|
|  19| Justin|1.7848230096318725E8|
|  35|   Emma|1.586013452313430...|
|null|   null|                null|
|  31|   null|2.904884966524742...|
+----+-------+--------------------+



### join
inner (domyslny)

In [183]:
people.join(other=employees, on='name', how='inner').show()

+-------+----+------+
|   name| age|salary|
+-------+----+------+
|Michael|null|  3000|
|   Andy|  30|  4500|
| Justin|  19|  3500|
+-------+----+------+



<b> Uwaga ogólna </b><br>
Join to stosunkowo popularna, ale kosztowna operacja.<br>
W sytuacji, kiedy jeden z łaczonych DataFramow jest znacznie mniejszy (w szczegolnosci na tyle mały, że w całości mieści się w pamięci), zaleca sie zastosowanie <i>broadcast hash join</i>.<br>
(Mała tabela zostanie zebrana do pamięci i wysłana do każdego noda).<br>
W niektórych przypadkach optymalizator sam za nas zdecyduje o zastosowaniu <i>broadcast hash join</i>.

In [184]:
from pyspark.sql.functions import broadcast
newPeople.join(broadcast(spark.createDataFrame([Row(age=20, name='Greg')])), on='name').show()

+----+---+---+
|name|age|age|
+----+---+---+
|Greg| 40| 20|
|Greg| 32| 20|
+----+---+---+



> **TODO**: Spośród osób (`people`, `newPeople`, `people_txt`), dla których mamy informacje o zarobkach (zbiór `employees`). Ile zarabia najmłodsza osoba?

In [195]:
cols= people.columns
union_people = people.union(people_txt.select(cols)).union(newPeople)
min_age = union_people.select(f.min("age")).collect()[0][0]
union_people.where(union_people.age == min_age).join(employees, on="name").show()


+------+----+------+
|  name| age|salary|
+------+----+------+
|Justin|19.0|  3500|
|Justin|19.0|  3500|
+------+----+------+



In [196]:
df_zad = union_people.dropna(subset="age").join(employees, on="name")


In [199]:
df_zad = df_zad.withColumn("increase", df_zad.salary * (0.001 * df_zad.age))
df_zad.show()

+-------+----+------+-------+--------+
|   name| age|salary|increas|increase|
+-------+----+------+-------+--------+
|   Andy|30.0|  4500|  135.0|   135.0|
| Justin|19.0|  3500|   66.5|    66.5|
|Michael|29.0|  3000|   87.0|    87.0|
|   Andy|30.0|  4500|  135.0|   135.0|
| Justin|19.0|  3500|   66.5|    66.5|
+-------+----+------+-------+--------+



In [200]:
df_zad.select(f.sum("increase").alias("bonus_cost")).show()

+----------+
|bonus_cost|
+----------+
|     490.0|
+----------+



> **TODO**: Dla każdego pracownika (`employees`), dla którego mamy informacje o wieku (`people`, `newPeople`, `people_txt`). Dodaj do pensji 0.1% za każdy rok życia. Oblicz koszt takiego 'bonusu urodzinowego' dla pracodawcy. 

### groupBy

In [ ]:
newPeople.groupBy()

 Przez GroupedData mamy dostep do takich funkcji jak:<br>
 avg, max, min, sum, count, agg <br>
 (dla wygody, do funkcji 'agg' mamy tez dostep bezposrednio na DataFrame)

In [201]:
newPeople.groupBy().max().show()

+--------+
|max(age)|
+--------+
|      40|
+--------+



In [202]:
newPeople.groupBy('name').count().show()

+-----+-----+
| name|count|
+-----+-----+
| Greg|    2|
| null|    2|
|Alice|    1|
|Peter|    2|
+-----+-----+



In [203]:
newPeople.groupBy('name').agg(f.min('age').alias('min_age'), f.max('age').alias('max_age'),\
                              f.count('name').alias('n_people')).show()

+-----+-------+-------+--------+
| name|min_age|max_age|n_people|
+-----+-------+-------+--------+
| Greg|     32|     40|       2|
| null|     33|     33|       0|
|Alice|     24|     24|       1|
|Peter|     32|     32|       2|
+-----+-------+-------+--------+



In [204]:
newPeople.agg(f.min('age'), f.max('age'), f.count('name')).show()

+--------+--------+-----------+
|min(age)|max(age)|count(name)|
+--------+--------+-----------+
|      24|      40|          5|
+--------+--------+-----------+



> **TODO**: Ile jest unikatowych (występujących tylko 1 raz) imion w połączonych zbiorach `people`, `newPeople` oraz `people_txt`?

In [205]:
counted_df = union_people.groupby("name").agg(f.count("name").alias("name_count"))
counted_df.where(counted_df.name_count == 1).count()

2

> **TODO**: Ile lat mają osoby, których imiona występują tylko raz w połączonych zbiorach `people`, `newPeople` oraz `people_txt`?

In [206]:
counted_df.where(counted_df.name_count == 1).join(union_people, on = "name").show()

+-----+----------+----+
| name|name_count| age|
+-----+----------+----+
|Alice|         1|24.0|
| Emma|         1|35.0|
+-----+----------+----+



****

#### Nowy DataFrame

In [207]:
import random

In [209]:
random.seed(123)

years = 10
names = ['Alice', 'Betty', 'Chris', 'Dan', 'Greg']
unique_names_count = len(names)
names = sorted(names*years)
year = [y for y in range(2000, 2000+years)]*len(names)
starting_salary = [round(random.gauss(4000, 1000),2) for i in range(unique_names_count)]
salary = [0 for i in range(years*unique_names_count)]
salary[::years] = starting_salary
for n in range(unique_names_count):
    for y in range(years-1):
        index = (years*n+1)+y
        salary[index] = round(salary[index-1]*(1+random.gauss(0.1,0.09)),2)

In [210]:
salaryHistory = spark.createDataFrame([Row(name=n, year=y, salary=s) for n,y,s in zip(names, year, salary)])
salaryHistory = salaryHistory.filter((salaryHistory['name'] != 'Greg') | (salaryHistory['year'] != 2006))
salaryHistory = salaryHistory.union(spark.createDataFrame([Row('Alice', 3000, 2000)]))

In [211]:
salaryHistory.show()

+-----+-------+----+
| name| salary|year|
+-----+-------+----+
|Alice|4404.23|2000|
|Alice|4780.34|2001|
|Alice|4881.72|2002|
|Alice|5280.86|2003|
|Alice|5976.68|2004|
|Alice|6320.14|2005|
|Alice|6685.07|2006|
|Alice|7816.44|2007|
|Alice|8599.32|2008|
|Alice|9503.99|2009|
|Betty|4138.01|2000|
|Betty|4404.94|2001|
|Betty|4911.61|2002|
|Betty|5265.17|2003|
|Betty|5687.88|2004|
|Betty|6033.45|2005|
|Betty|7179.65|2006|
|Betty|8133.84|2007|
|Betty|8766.33|2008|
|Betty|10516.8|2009|
+-----+-------+----+
only showing top 20 rows



****

> **TODO**: Ile razy powtarza się każde z imion w `salaryHistory`?

In [227]:
salaryHistory.groupBy("name").count().show()

+-----+-----+
| name|count|
+-----+-----+
|Chris|   10|
| Greg|    9|
|Betty|   10|
|  Dan|   10|
|Alice|   11|
+-----+-----+



> **TODO**: Na podstawie `salaryHistory` stworz tabelę zależności średniej, minimalnej i maksymalnej pensji od roku. Posortuj lata malejąco. Pensje podaj z dokładnością do pełnych wartości.

In [228]:
salaryHistory.groupby("year").agg(f.round(f.avg("salary")).alias("avg"), f.round(f.min("salary")).alias("min"), f.round(f.max("salary")).alias("max"))\
.orderBy(f.desc("year")).show()

+----+------+------+-------+
|year|   avg|   min|    max|
+----+------+------+-------+
|2009|8935.0|5838.0|10517.0|
|2008|7889.0|5597.0| 8766.0|
|2007|7497.0|5081.0| 8299.0|
|2006|7435.0|6685.0| 8418.0|
|2005|5997.0|4200.0| 7129.0|
|2004|5608.0|3778.0| 6672.0|
|2003|5283.0|3908.0| 6436.0|
|2002|5114.0|4174.0| 5887.0|
|2001|4515.0|4152.0| 4830.0|
|2000|3939.0|3000.0| 4404.0|
+----+------+------+-------+



### Window functions
**over**

Służy do obliczania agregowanych wartości w grupach definiowanych oknem (window).<br>
Zwraca wiele rekordow (tyle ile na wejsciu w grupie).

partitionBy - definiuje podział danych na okna<br>
orderBy - definiuje sortowanie wewnątrz każdego z okien<br>
Frame (rangeBetween/rowsBetween) - definiuje offset<br>

In [229]:
from pyspark.sql.window import Window

**partitionBy**

In [234]:
allPeople = spark.read.parquet(data_path+'allPeople.parquet')

In [235]:
# definicja 'okna'
myWindowSpec = Window.partitionBy('name')

In [236]:
# wywołanie funkcji na kazdym 'oknie'
allPeople.withColumn('nameCount', \
                     f.count(allPeople['name']).over(myWindowSpec)).show()

+----+-------+---------+
| age|   name|nameCount|
+----+-------+---------+
|40.0|   Greg|        2|
|32.0|   Greg|        2|
|null|   null|        0|
|33.0|   null|        0|
|null|Michael|        2|
|29.0|Michael|        2|
|24.0|  Alice|        1|
|35.0|   Emma|        1|
|30.0|   Andy|        2|
|30.0|   Andy|        2|
|19.0| Justin|        2|
|19.0| Justin|        2|
|32.0|  Peter|        2|
|null|  Peter|        2|
+----+-------+---------+



> **TODO**: Do zbioru `salaryHistory` dodaj kolumnę 'avgSalaryDiff', która będzie zawierała różnicę pomiedzy pensją z danego roku, a średnią pensją osoby na przestrzeni wszytskich lat.

In [237]:
myWindowSpec = Window.partitionBy("name")
salaryHistory.withColumn("avgSalaryDiv", salaryHistory.salary - f.avg(salaryHistory.salary).over(myWindowSpec))\
.withColumn("avgSalary", f.avg(salaryHistory.salary).over(myWindowSpec)).show()

+-----+--------+----+-------------------+-----------------+
| name|  salary|year|       avgSalaryDiv|        avgSalary|
+-----+--------+----+-------------------+-----------------+
|Chris| 3601.42|2000| -3293.245000000001|6894.665000000001|
|Chris| 4407.79|2001| -2486.875000000001|6894.665000000001|
|Chris|  5713.7|2002| -1180.965000000001|6894.665000000001|
|Chris|  6435.6|2003| -459.0650000000005|6894.665000000001|
|Chris| 6671.86|2004| -222.8050000000012|6894.665000000001|
|Chris| 7129.19|2005| 234.52499999999873|6894.665000000001|
|Chris| 8417.84|2006| 1523.1749999999993|6894.665000000001|
|Chris| 8155.56|2007| 1260.8949999999995|6894.665000000001|
|Chris| 8239.77|2008| 1345.1049999999996|6894.665000000001|
|Chris|10173.92|2009|  3279.254999999999|6894.665000000001|
| Greg| 4226.89|2000| -323.7622222222226|4550.652222222223|
| Greg| 4151.85|2001|-398.80222222222255|4550.652222222223|
| Greg| 4174.36|2002|-376.29222222222324|4550.652222222223|
| Greg| 3908.07|2003| -642.5822222222228

**partitionBy + orderBy**

In [238]:
# przykład: rank
# - musimy zdefiniować dodatkowo sortowanie wewnątrz każdej z grup
# - zwraca lp dla kolejnych rekordów posortowanych według zadanych kolumn
windowSpec = Window.partitionBy(salaryHistory['name']).orderBy(salaryHistory['year'])
ranked = f.rank().over(windowSpec)
salaryHistory.withColumn('ranked', ranked).show()

+-----+--------+----+------+
| name|  salary|year|ranked|
+-----+--------+----+------+
|Chris| 3601.42|2000|     1|
|Chris| 4407.79|2001|     2|
|Chris|  5713.7|2002|     3|
|Chris|  6435.6|2003|     4|
|Chris| 6671.86|2004|     5|
|Chris| 7129.19|2005|     6|
|Chris| 8417.84|2006|     7|
|Chris| 8155.56|2007|     8|
|Chris| 8239.77|2008|     9|
|Chris|10173.92|2009|    10|
| Greg| 4226.89|2000|     1|
| Greg| 4151.85|2001|     2|
| Greg| 4174.36|2002|     3|
| Greg| 3908.07|2003|     4|
| Greg| 3778.21|2004|     5|
| Greg| 4199.99|2005|     6|
| Greg| 5081.34|2007|     7|
| Greg| 5597.06|2008|     8|
| Greg|  5838.1|2009|     9|
|Betty| 4138.01|2000|     1|
+-----+--------+----+------+
only showing top 20 rows



In [239]:
salaryHistory_tmp = salaryHistory.filter(salaryHistory.name.isin('Alice', 'Greg'))

In [240]:
# rank, dense_rank, percent_rank, row_number
windowSpec = Window.partitionBy('name').orderBy('year')
ranked = (f.rank()).over(windowSpec)
dense_rank = (f.dense_rank()).over(windowSpec)
percent_rank = (f.percent_rank()).over(windowSpec)
row_number = (f.row_number()).over(windowSpec)
salaryHistory_tmp.withColumn('ranked', ranked).withColumn('dense_rank', dense_rank).\
withColumn('percent_rank', percent_rank).withColumn('row_number', row_number).\
show()

+-----+-------+----+------+----------+------------+----------+
| name| salary|year|ranked|dense_rank|percent_rank|row_number|
+-----+-------+----+------+----------+------------+----------+
| Greg|4226.89|2000|     1|         1|         0.0|         1|
| Greg|4151.85|2001|     2|         2|       0.125|         2|
| Greg|4174.36|2002|     3|         3|        0.25|         3|
| Greg|3908.07|2003|     4|         4|       0.375|         4|
| Greg|3778.21|2004|     5|         5|         0.5|         5|
| Greg|4199.99|2005|     6|         6|       0.625|         6|
| Greg|5081.34|2007|     7|         7|        0.75|         7|
| Greg|5597.06|2008|     8|         8|       0.875|         8|
| Greg| 5838.1|2009|     9|         9|         1.0|         9|
|Alice|4404.23|2000|     1|         1|         0.0|         1|
|Alice| 3000.0|2000|     1|         1|         0.0|         2|
|Alice|4780.34|2001|     3|         2|         0.2|         3|
|Alice|4881.72|2002|     4|         3|         0.3|    

Inne warte uwagi funkcje: <br>
ntile, cume_dist, first, lag, lead

> **TODO**: Dla zbioru `salaryHistory`, porównaj pensje ludzi pomiedzy najwcześniejszym i najpóźniejszym rokiem ich pracy.

In [243]:
windowSpec = Window.partitionBy("name").orderBy("year")
windowSpec_desc = Window.partitionBy("name").orderBy(f.desc("year"))

In [244]:
salaryHistory.withColumn("first_sal", f.first("salary").over(windowSpec))\
.withColumn("last_sal", f.first("salary").over(windowSpec_desc))\
.withColumn("salary_diff", f.first("salary").over(windowSpec) - f.first("salary").over(windowSpec_desc)).show()


+-----+--------+----+---------+--------+------------------+
| name|  salary|year|first_sal|last_sal|       salary_diff|
+-----+--------+----+---------+--------+------------------+
|Chris|10173.92|2009|  3601.42|10173.92|           -6572.5|
|Chris| 8239.77|2008|  3601.42|10173.92|           -6572.5|
|Chris| 8155.56|2007|  3601.42|10173.92|           -6572.5|
|Chris| 8417.84|2006|  3601.42|10173.92|           -6572.5|
|Chris| 7129.19|2005|  3601.42|10173.92|           -6572.5|
|Chris| 6671.86|2004|  3601.42|10173.92|           -6572.5|
|Chris|  6435.6|2003|  3601.42|10173.92|           -6572.5|
|Chris|  5713.7|2002|  3601.42|10173.92|           -6572.5|
|Chris| 4407.79|2001|  3601.42|10173.92|           -6572.5|
|Chris| 3601.42|2000|  3601.42|10173.92|           -6572.5|
| Greg|  5838.1|2009|  4226.89|  5838.1|          -1611.21|
| Greg| 5597.06|2008|  4226.89|  5838.1|          -1611.21|
| Greg| 5081.34|2007|  4226.89|  5838.1|          -1611.21|
| Greg| 4199.99|2005|  4226.89|  5838.1|

**partitionBy + orderBy + rangeBetween/rowsBetween**

In [245]:
# przykład: średnia ruchoma
windowSpec = Window.partitionBy(salaryHistory['name']).orderBy(salaryHistory['year'])\
.rowsBetween(-1,1)
movingAvg = (f.avg(salaryHistory['salary'])).over(windowSpec)
salaryHistory.withColumn('movingAvg', movingAvg).show()

+-----+--------+----+------------------+
| name|  salary|year|         movingAvg|
+-----+--------+----+------------------+
|Chris| 3601.42|2000|          4004.605|
|Chris| 4407.79|2001| 4574.303333333333|
|Chris|  5713.7|2002|           5519.03|
|Chris|  6435.6|2003|           6273.72|
|Chris| 6671.86|2004| 6745.549999999999|
|Chris| 7129.19|2005| 7406.296666666666|
|Chris| 8417.84|2006| 7900.863333333334|
|Chris| 8155.56|2007| 8271.056666666667|
|Chris| 8239.77|2008| 8856.416666666666|
|Chris|10173.92|2009| 9206.845000000001|
| Greg| 4226.89|2000| 4189.370000000001|
| Greg| 4151.85|2001| 4184.366666666668|
| Greg| 4174.36|2002| 4078.093333333333|
| Greg| 3908.07|2003|3953.5466666666666|
| Greg| 3778.21|2004|           3962.09|
| Greg| 4199.99|2005|           4353.18|
| Greg| 5081.34|2007| 4959.463333333333|
| Greg| 5597.06|2008|            5505.5|
| Greg|  5838.1|2009|           5717.58|
|Betty| 4138.01|2000|          4271.475|
+-----+--------+----+------------------+
only showing top

In [246]:
# przykład: średnia ze wszystkich rekordów do aktualnego włącznie
windowSpec = Window.partitionBy(salaryHistory['name']).orderBy(salaryHistory['year']).\
rowsBetween(Window.unboundedPreceding,Window.currentRow)
movingAvg = (f.avg(salaryHistory['salary'])).over(windowSpec)
salaryHistory.withColumn('movingAvg', movingAvg).show()

+-----+--------+----+------------------+
| name|  salary|year|         movingAvg|
+-----+--------+----+------------------+
|Chris| 3601.42|2000|           3601.42|
|Chris| 4407.79|2001|          4004.605|
|Chris|  5713.7|2002| 4574.303333333333|
|Chris|  6435.6|2003|5039.6275000000005|
|Chris| 6671.86|2004|5366.0740000000005|
|Chris| 7129.19|2005| 5659.926666666667|
|Chris| 8417.84|2006| 6053.914285714287|
|Chris| 8155.56|2007| 6316.620000000001|
|Chris| 8239.77|2008| 6530.303333333334|
|Chris|10173.92|2009| 6894.665000000001|
| Greg| 4226.89|2000|           4226.89|
| Greg| 4151.85|2001| 4189.370000000001|
| Greg| 4174.36|2002| 4184.366666666668|
| Greg| 3908.07|2003|4115.2925000000005|
| Greg| 3778.21|2004|          4047.876|
| Greg| 4199.99|2005| 4073.228333333334|
| Greg| 5081.34|2007| 4217.244285714286|
| Greg| 5597.06|2008|4389.7212500000005|
| Greg|  5838.1|2009| 4550.652222222223|
|Betty| 4138.01|2000|           4138.01|
+-----+--------+----+------------------+
only showing top

In [247]:
salaryHistory.withColumn('movingAvg', movingAvg).filter("name == 'Alice'").show()

+-----+-------+----+------------------+
| name| salary|year|         movingAvg|
+-----+-------+----+------------------+
|Alice|4404.23|2000|           4404.23|
|Alice| 3000.0|2000|          3702.115|
|Alice|4780.34|2001| 4061.523333333333|
|Alice|4881.72|2002|         4266.5725|
|Alice|5280.86|2003|           4469.43|
|Alice|5976.68|2004| 4720.638333333333|
|Alice|6320.14|2005| 4949.138571428572|
|Alice|6685.07|2006|           5166.13|
|Alice|7816.44|2007| 5460.608888888889|
|Alice|8599.32|2008|5774.4800000000005|
|Alice|9503.99|2009| 6113.526363636364|
+-----+-------+----+------------------+



In [248]:
# podobny efekt uzyskamy poniższym zapytaniemm. 
# Różnica: rekordy w jednej grupie (imię, rok) nie zostaną rozdzielone 
import sys
windowSpec = Window.partitionBy(salaryHistory['name']).orderBy(salaryHistory['year']).\
rangeBetween(-sys.maxsize,0)
movingAvg = (f.avg(salaryHistory['salary'])).over(windowSpec)
salaryHistory.withColumn('movingAvg', movingAvg).show()

+-----+--------+----+------------------+
| name|  salary|year|         movingAvg|
+-----+--------+----+------------------+
|Chris| 3601.42|2000|           3601.42|
|Chris| 4407.79|2001|          4004.605|
|Chris|  5713.7|2002| 4574.303333333333|
|Chris|  6435.6|2003|5039.6275000000005|
|Chris| 6671.86|2004|5366.0740000000005|
|Chris| 7129.19|2005| 5659.926666666667|
|Chris| 8417.84|2006| 6053.914285714287|
|Chris| 8155.56|2007| 6316.620000000001|
|Chris| 8239.77|2008| 6530.303333333334|
|Chris|10173.92|2009| 6894.665000000001|
| Greg| 4226.89|2000|           4226.89|
| Greg| 4151.85|2001| 4189.370000000001|
| Greg| 4174.36|2002| 4184.366666666668|
| Greg| 3908.07|2003|4115.2925000000005|
| Greg| 3778.21|2004|          4047.876|
| Greg| 4199.99|2005| 4073.228333333334|
| Greg| 5081.34|2007| 4217.244285714286|
| Greg| 5597.06|2008|4389.7212500000005|
| Greg|  5838.1|2009| 4550.652222222223|
|Betty| 4138.01|2000|           4138.01|
+-----+--------+----+------------------+
only showing top

In [249]:
salaryHistory.withColumn('movingAvg', movingAvg).filter("name == 'Alice'").show()

+-----+-------+----+------------------+
| name| salary|year|         movingAvg|
+-----+-------+----+------------------+
|Alice|4404.23|2000|          3702.115|
|Alice| 3000.0|2000|          3702.115|
|Alice|4780.34|2001| 4061.523333333333|
|Alice|4881.72|2002|         4266.5725|
|Alice|5280.86|2003|           4469.43|
|Alice|5976.68|2004| 4720.638333333333|
|Alice|6320.14|2005| 4949.138571428572|
|Alice|6685.07|2006|           5166.13|
|Alice|7816.44|2007| 5460.608888888889|
|Alice|8599.32|2008|5774.4800000000005|
|Alice|9503.99|2009| 6113.526363636364|
+-----+-------+----+------------------+



###############################################################################################################

# Zadania 2

1. Ile lat mają osoby, których imiona występują tylko raz w połączonych zbiorach `people`, `newPeople` oraz `people_txt`? Rozwiązując problem zastosuj window functions.

2. Czy komukolwiek obniżyła się pensja w stosunku do roku poprzedniego? <br>
a. Ile osób było kiedykolwiek w takiej sytuacji?<br>
b. Jaki jest rozkład częstości takich przypadków w zależności od roku?

3. Oblicz różnicę w pensjach w stosuku do <br>
a. najwyższej pensji danej osoby<br> 
b. drugiej najwyższej pensji danej osoby<br>

###############################################################################################################

### SQL

Spark wspiera ANSI SQL 2003 (SQL3)

Aby użyć zbioru danych w zapytaniu SQL musimy go najpierw zarejestrować.

In [ ]:
salaryHistory.createOrReplaceTempView('salaryHistory')
# salaryHistory.registerTempTable('salaryHistorySQL') - wersja dla sparka 1.x
# view NIE jest 'persisted in memory', to nadal nie jest akcja

In [ ]:
spark.sql('select * from salaryHistory limit 2').show()

In [ ]:
type(spark.sql('select * from salaryHistory limit 2'))

In [ ]:
spark.sql('select name, avg(salary) avg_sal from salaryHistory group by name').show()

In [ ]:
spark.sql('select name, salary, avg(salary) over (partition by name) avg_sal from salaryHistory').show()

In [ ]:
spark.catalog.dropTempView("salaryHistory")

### UDFs
Poza funkcjami dostępnymi w module pyspark.sql.functions, można też tworzyć własne (User Defined Functions) i używać ich w zapytaniach SparkSQL <br>
<i>Uwaga:</i> Po UDFy sięgamy w ostateczności - optymalizacja <br>
<i>Uwaga:</i> UDF napisany w Pythonie (lub R) bedzie mial gorszy performance niż UDF napisany w Scali lub Javie. Ale możemy nasz UDF napisać w Scali/Javie i zarejestrować do użycia w Pythonie. <br>
Blogpost o tym jak w pySparku używać UDF napisanej w scali: <br>
https://medium.com/@ingwbaa/using-scala-udfs-in-pyspark-b70033dd69b9 <br>
Blogpost o ulepszeniach w UDF (pandas UDF) dla pySparka od Sparku 2.3: <br>
https://databricks.com/blog/2017/10/30/introducing-vectorized-udfs-for-pyspark.html

In [ ]:
from pyspark.sql.types import IntegerType, StringType, FloatType

In [ ]:
def udfPower3(value):
    return(value**3)

In [ ]:
udfPower3(3.14)

Aby stosować zdefiniowaną funkcję, musimy ja zarejestrować. (Driver roześle funkcje do wszystkich egzekutorów)

In [ ]:
power3 = f.udf(udfPower3, FloatType())

In [ ]:
salaryHistory.select(power3(salaryHistory.salary)).show()

### Spark wspiera Hive

Spark SQL wspiera HiveQL. <br>
Spark SQL wspiera rownież wczytawanie/zapisywanie danych z/do Apache Hive.<br>

Wiecej informacji na temat integracji z Hive:<br>
https://spark.apache.org/docs/latest/sql-programming-guide.html#compatibility-with-apache-hive <br>
https://spark.apache.org/docs/latest/sql-programming-guide.html#hive-tables